In [1]:
import os
import time
import re
import random as rd
import pandas as pd
import numpy as np
import collections
from tqdm import tqdm
import pickle

from konlpy.tag import Mecab

import multiprocessing as mp
import parmap

c:\Users\user\.virtualenvs\project-D53q-EhT\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_pickle("../data/preprocessed_total.pkl")
df

,Query,Title,Text,News
0,더불어민주당,막대기도 당선될 판 진중권 네거티브 민주당 비판,진중권 전 동양대 교수 이미지출처연합뉴스 AD 원본보기 아이콘아시아경제 황수미 기자...,asiae
1,더불어민주당,1년만에 뒤바뀐 공수민주당 사죄 국민의힘 여론조사가 민심,이낙연 분노와 실망 아프도록 잘 안다 반성하고 혁신주호영 여론조사 2030 차이 민...,asiae
2,더불어민주당,종합민주당 오세훈 내곡동 특혜 의혹에 거짓말이 거짓말을 낳아 맹공,박영선 TV 토론서 오세훈 내곡동 특혜 의혹 두고 공세47 서울시장 보궐선거 더불어...,asiae
3,더불어민주당,박형준 안민석 더불어민주당 의원진보 유튜버 등 4명 부산지검 고발,국민의힘 캠프 부동산 투기 전혀 없다 안 의원 후보부인 부동산 복부인 발언에 반박...,asiae
4,더불어민주당,포토 더불어민주당 원내대책회의,가장 많이 읽힌 뉴스를 제공합니다 집계 기준에 따라 최대 3일 전 기사까지 제공될 ...,asiae
...,...,...,...,...
181819,더불어민주당,여론톡톡 김기현 책임론 확산이낙연이준석 손잡나,AD 진행 김영수 앵커 출연 김형준 배재대 석좌교수 배종찬 인사이트 K 연구소장...,YTN
181820,더불어민주당,홍익표 쌍특검 8일 본회의국정조사도 연내 처리,AD더불어민주당 홍익표 원내대표는 쌍특검 법안과 예산안은 오는 8일 본회의에 올리고...,YTN
181821,더불어민주당,손학규 병립형 회귀 큰 후퇴이재명 약속 지켜야,AD손학규 전 바른미래당 대표는 선거제 개편 논의와 관련해 연동형 비례제는 더불어민...,YTN
181822,더불어민주당,민주 송영길 구속영장 청구 검찰 독재정권의 총선 전략,AD더불어민주당은 검찰이 전당대회 돈봉투 의혹에 연루된 송영길 전 대표에게 구속영장...,YTN


In [3]:
mecab = Mecab(dicpath='C:/mecab/mecab-ko-dic')

In [4]:
tqdm.pandas()   # progress_apply를 사용하기 위한 설정
num_workers = mp.cpu_count()    # 병렬 처리를 위한 worker 수 설정
print(f'num_workers: {num_workers}')    

num_workers: 20


In [5]:
# 형태소 분석

# df["Tokenized"] = df["Text"].progress_apply(lambda x: mecab.morphs(x))
# parmap.map 함수를 사용하여 mecab.morphs 함수를 병렬로 적용
df["Tokenized"] = parmap.map(mecab.morphs, df["Text"], pm_pbar=True, pm_processes=num_workers)  # parmap.map 함수를 사용하여 mecab.morphs 함수를 병렬로 적용

# 열 인덱스 지정
df = df[["Query", "Title", "Text", "News", "Tokenized"]]

  0%|          | 0/181824 [00:04<?, ?it/s]


In [1]:
import pandas as pd
import pickle


In [2]:
df = pd.read_pickle("../data/tokenized_total.pkl")
df

,Query,Title,Text,News,Tokenized
0,더불어민주당,막대기도 당선될 판 진중권 네거티브 민주당 비판,진중권 전 동양대 교수 이미지출처연합뉴스 AD 원본보기 아이콘아시아경제 황수미 기자...,asiae,"[진중권, 전, 동양, 대, 교수, 이미지, 출처, 연합뉴스, AD, 원본, 보, ..."
1,더불어민주당,1년만에 뒤바뀐 공수민주당 사죄 국민의힘 여론조사가 민심,이낙연 분노와 실망 아프도록 잘 안다 반성하고 혁신주호영 여론조사 2030 차이 민...,asiae,"[이낙연, 분노, 와, 실망, 아프, 도록, 잘, 안, 다, 반성, 하, 고, 혁신..."
2,더불어민주당,종합민주당 오세훈 내곡동 특혜 의혹에 거짓말이 거짓말을 낳아 맹공,박영선 TV 토론서 오세훈 내곡동 특혜 의혹 두고 공세47 서울시장 보궐선거 더불어...,asiae,"[박영선, TV, 토론, 서, 오세훈, 내곡동, 특혜, 의혹, 두, 고, 공세, 4..."
3,더불어민주당,박형준 안민석 더불어민주당 의원진보 유튜버 등 4명 부산지검 고발,국민의힘 캠프 부동산 투기 전혀 없다 안 의원 후보부인 부동산 복부인 발언에 반박...,asiae,"[국민의힘, 캠프, 부동산, 투기, 전혀, 없, 다, 안, 의원, 후보, 부인, 부..."
4,더불어민주당,포토 더불어민주당 원내대책회의,가장 많이 읽힌 뉴스를 제공합니다 집계 기준에 따라 최대 3일 전 기사까지 제공될 ...,asiae,"[가장, 많이, 읽힌, 뉴스, 를, 제공, 합니다, 집계, 기준, 에, 따라, 최대..."
...,...,...,...,...,...
181819,더불어민주당,여론톡톡 김기현 책임론 확산이낙연이준석 손잡나,AD 진행 김영수 앵커 출연 김형준 배재대 석좌교수 배종찬 인사이트 K 연구소장...,YTN,"[AD, 진행, 김영수, 앵커, 출연, 김형준, 배재, 대, 석좌, 교수, 배종찬,..."
181820,더불어민주당,홍익표 쌍특검 8일 본회의국정조사도 연내 처리,AD더불어민주당 홍익표 원내대표는 쌍특검 법안과 예산안은 오는 8일 본회의에 올리고...,YTN,"[AD, 더불어민주당, 홍익표, 원내대표, 는, 쌍, 특검, 법안, 과, 예산안, ..."
181821,더불어민주당,손학규 병립형 회귀 큰 후퇴이재명 약속 지켜야,AD손학규 전 바른미래당 대표는 선거제 개편 논의와 관련해 연동형 비례제는 더불어민...,YTN,"[AD, 손학규, 전, 바른, 미래, 당, 대표, 는, 선거제, 개편, 논의, 와,..."
181822,더불어민주당,민주 송영길 구속영장 청구 검찰 독재정권의 총선 전략,AD더불어민주당은 검찰이 전당대회 돈봉투 의혹에 연루된 송영길 전 대표에게 구속영장...,YTN,"[AD, 더불어민주당, 은, 검찰, 이, 전당, 대회, 돈, 봉투, 의혹, 에, 연..."
